# Login HuggingFace to Upload Model

In [1]:
!git config --global credential.helper store

from huggingface_hub import notebook_login
notebook_login()

# 1. Training

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
!nvidia-smi

Mon Mar  7 15:47:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   60C    P0    44W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   55C    P0    41W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [4]:
def print_encoded(encoded_inputs):
    for ts in encoded_inputs:
        print(ts, encoded_inputs[ts])

## 1.1 Prepare-Datasets

In [6]:
import transformers
print(transformers.__version__)

4.17.0


In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

In [8]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

# raw_datasets = load_dataset("scb_mt_enth_2020", name="then")
metric = load_metric("sacrebleu")

In [9]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path)
    list_5M = df_5M.to_dict('records')[:n_row]
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [10]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 100000)

In [11]:
raw_datasets

,translation,subdataset
0,"{'th': 'ขอบคุณ แต่ ฉัน ไม่ มี เวลา พอ', 'en': ...",LST_Corpus
1,"{'th': 'โอ้ , คุณ รู้ , เมื่อ ฉัน ตื่น ฉัน จะ ...",LST_Corpus
2,{'th': 'หน้าที่ ของ ตำรวจ คือ การ ปกป้อง ประชา...,LST_Corpus
3,"{'th': 'มัน เป็นความ โอหัง ของ เพศ หญิง .', 'e...",LST_Corpus
4,"{'th': 'โปรด คอย ดู อย่า เผลอ', 'en': 'please ...",LST_Corpus
...,...,...
99995,"{'th': 'สอน วจนะ ของ พระเจ้า', 'en': 'preach t...",LST_Corpus
99996,"{'th': 'จาน บิน', 'en': 'flying saucer'}",LST_Corpus
99997,{'th': 'คุณ มา ที่ รัฐ นี้ แล้ว จำนวน กี่ ครั้...,LST_Corpus
99998,"{'th': 'ทาง แยก ไป ทางซ้าย', 'en': 'the left f...",LST_Corpus


In [12]:
cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.2)

In [13]:
cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 20000
    })
})

In [14]:
import datasets, random
import pandas as pd
from IPython.display import display, HTML

def show_random_element(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than dataset"
    picks = []
    for n_ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks: # random inside
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_element(cut_datasets['train'])

,translation,subdataset
0,"{'en': 'how much will it cost to fix it?', 'th': 'ซ่อม มัน ราคา เท่า ไหร่ ?'}",LST_Corpus
1,"{'en': 'i have a previous engagement', 'th': 'ฉัน มี นัด ไว้ ก่อน แล้ว'}",LST_Corpus
2,"{'en': 'when she heard that her husband was at the gambling house , she looked sour at once .', 'th': 'เมื่อ เธอ ได้ยิน ว่า สา มีอยู่ ที่ บ่อน การพนัน เธอ ก็มี ใบ หน้า บูดบึ้ง ทันที'}",LST_Corpus
3,"{'en': 'the manager and his staff are in conference with the company 's consultants .', 'th': 'ผู้จัดการ และ ลูกน้อง ของ เขา กำลัง อยู่ ในระหว่าง การ หารือ กับ ที่ปรึกษา ของ บริษัท'}",LST_Corpus
4,"{'en': 'do you have anything for insect bites', 'th': 'คุณ มี อะไร สำหรับ การ ถูก แมลง กัด ต่อย ไหม'}",LST_Corpus


In [16]:
cut_datasets['train'][0:2]

{'translation': [{'en': 'noi struck a match to light a fire in the grill .',
   'th': 'หน่อย จุด ไม้ขีด เพื่อ จุด ไฟ ใน เตา ปิ้ง'},
  {'en': 'how neatly the food is arranged on the dishes',
   'th': 'อาหาร ช่าง ดี ต่อ สุภาพ อะไร เช่นนี้'}],
 'subdataset': ['LST_Corpus', 'LST_Corpus']}

#### Metric

In [17]:
# metric

In [18]:
metric = load_metric("sacrebleu")
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## 1.2 Preprocessing the Data

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [20]:
# Set Source and Target Language

if 'mbart' in model_checkpoint:
    tokenizer.src_lang = 'th'
    tokenizer.tgt_lang = 'en'

In [21]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [32]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    targets= [ex[target_lang] for ex in examples['translation']]
    
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)

    with tokenizer.as_target_tokenizer():
        try:
            labels = tokenizer(targets, max_length=max_target_len, truncation=True, padding=True)
        except:
            print('target: ', target)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [33]:
len(cut_datasets['train'])

80000

In [34]:
cut_datasets['train']

Dataset({
    features: ['translation', 'subdataset'],
    num_rows: 80000
})

In [35]:
# for i in range(len(cut_datasets['train'])):
#     print(i)
#     if i >= 3000:
#         break
#     preprocess_function(cut_datasets['train'][i])

In [36]:
tokenized_datasets = cut_datasets.map(preprocess_function, batched=True)

  0%|          | 0/80 [00:00<?, ?ba/s]

NameError: name 'target' is not defined

In [ ]:
tokenized_datasets

In [ ]:
tokenized_datasets

## 1.3 Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(tokenized_datasets)

In [ ]:
batch_size = 32
model_name = cut_datasets['train'].split('/')[-1]
args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-5k-{source_lang}-to-{target_lang}',
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size//4,
    weight_decay =0.01,
    save_total_limit = 3,
    num_train_epochs=1,
    predict_with_generate=True,
    dataloader_num_workers=32, 
    fp16=True,
    push_to_hub = False,
)

#### Create Data collator

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels= [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    print('preds', preds)
    print('labels', labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

## 1.4 *train time!*

In [ ]:
trainer.train()

In [ ]:
?trainer.push_to_hub

In [ ]:
!git lfs install
trainer.push_to_hub()

In [ ]:
!ls

When Loading the model


```
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```



# 2. Using Translation Model

### Using AutoModel (With model selection)

In [ ]:
?AutoModelForSeq2SeqLM.

In [ ]:
tokenized_datasets['test']['labels'][0]

In [ ]:
# metric = load_metric("bleu")
# fake_preds = [["hello"],["there"], ["general"], ['kenobi']]
# fake_labels = [["hello"],["there"], ["general"], ['kenobi']]
# # fake_labels = [["hello there"], ["general kenobi"]]
# metric.compute(predictions=fake_preds, 
#                references=fake_labels)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
model_for_predict = "./opus-mt-th-en-finetuned-5k-th-to-en"
# model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

In [ ]:
!git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

In [ ]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [ ]:
for i in range(10):
    text_ = tokenized_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text:',  thai_text)
    print('real_text: ', real_text)
    print('pred_text: ', pred_text)
    print('pre_train: ', pred_text_pt)
    print()